In [1]:
import os
import json
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

data_dir = os.path.join('..','..','data')

with open(os.path.join(data_dir,'shapefiles','geojson','Nakhon-subdistricts.geojson')) as f:
    shapefile = json.load(f)

### Detected Breeding Sites

In [2]:
df_detect.head(3)

,district,subdist,bin,bowl,bucket,cup,jar,pottedplant,tire,vase,total
date,,,,,,,,,,,
2016-03-01,ร่อนพิบูลย์,ร่อนพิบูลย์,59,0,0,19,46,0,47,2,173
2016-03-01,ทุ่งสง,กะปาง,266,1,8,18,71,1,160,6,531
2016-03-01,ชะอวด,ชะอวด,329,1,14,37,46,7,200,5,639


### Larval Surveys

In [3]:
df_survey = pd.read_csv('../data/breeding-sites/larval-survey.csv') 
df_survey = df_survey.replace(0, np.nan)
df_survey = df_survey.dropna(axis=0, how='any')
df_survey = df_survey.reset_index(drop=True)
df_survey = df_survey.loc[df_survey['province'] == 'นครศรีธรรมราช']
df_survey = df_survey.drop('province', axis=1)
df_survey['date'] = pd.to_datetime(df_survey['date'], format='%Y-%m')
df_survey = df_survey.set_index('date')
df_survey = df_survey.sort_index()
df_survey.head(3)

,district,subdist,village,hi,ci,bi
date,,,,,,
2011-02-01,ทุ่งสง,นาหลวงเสน,ประดู่,25.0,2.77,25.0
2011-02-01,ฉวาง,นาแว,ทุ่งกระจูด,15.0,3.06,15.0
2011-02-01,ฉวาง,นาแว,ควนสวรรค์,17.5,2.86,17.5


### Area

In [4]:
df_area = pd.read_csv(os.path.join('..', 'data', 'area.csv'))
df_area.head(3)

,province,district,subdist,area
0,นครศรีธรรมราช,พรหมคีรี,ทอนหงส์,1070.58
1,นครศรีธรรมราช,ท่าศาลา,โมคลาน,341.44
2,นครศรีธรรมราช,เมืองนครศรีธรรมราช,ปากพูน,856.09


### Population

In [5]:
df_population.head(3)

,province,district,subdist,n_villages,population,area
0,นครศรีธรรมราช,เมืองนครศรีธรรมราช,ในเมือง,0,43327,NaN
1,นครศรีธรรมราช,เมืองนครศรีธรรมราช,ท่าวัง,0,17346,NaN
2,นครศรีธรรมราช,เมืองนครศรีธรรมราช,คลัง,0,17568,NaN


### Dengue Cases

In [6]:
df_dengue_cases.head(3)

,province,district,subdist,cases
0,นครศรีธรรมราช,เมืองนครศรีธรรมราช,ในเมือง,84
1,นครศรีธรรมราช,เมืองนครศรีธรรมราช,ท่าวัง,43
2,นครศรีธรรมราช,เมืองนครศรีธรรมราช,คลัง,24


In [7]:
def survey_timeseries(larval_surveys, index):
    BI_entry = {}
    year = 2011
    while(year < 2018):
        if year in larval_surveys.index.year.values:
            BI_entry[str(year)] = int(larval_surveys[str(year)]['bi'].mean())
        else:
            BI_entry[str(year)] = 0
        year += 1
    return BI_entry    

In [8]:
x = []
cnt = 0
for feature in shapefile['features']:
    
    prop = feature['properties']
    
    province = prop['PV_TN']
    district = prop['AP_TN']
    subdist = prop['TB_TN']
    
    brd_site_counts = df_detect.loc[
        (df_detect['district'] == district) &
        (df_detect['subdist'] == subdist)
    ]['total']
    
    larval_surveys = df_survey.loc[
        (df_survey['subdist'] == subdist)
    ]
    
    area = df_area.loc[
        (df_area['province'] == province) & 
        (df_area['district'] == district) &
        (df_area['subdist'] == subdist)
    ]['area']
    
    population = df_population.loc[
        (df_population['province'] == province) & 
        (df_population['district'] == district) &
        (df_population['subdist'] == subdist)
    ]['population']
    
    
    dengue_cases = df_dengue_cases.loc[
        (df_dengue_cases['province'] == province) & 
        (df_dengue_cases['district'] == district) &
        (df_dengue_cases['subdist'] == subdist)
    ]['cases']
    
    area = int(np.squeeze(area)) if len(area) == 1 else 1
    population = int(np.squeeze(population)) if len(population) == 1 else 0
    dengue_cases = int(np.squeeze(dengue_cases)) if len(dengue_cases) == 1 else 0
    brd_site_counts = int(np.squeeze(brd_site_counts)) if len(brd_site_counts) == 1 else 0
    
#     larval_surveys = int(larval_surveys.mean()) if len(larval_surveys) > 1 else 0

    BI_entry = survey_timeseries(larval_surveys, 'bi')
    
    feature['properties']['Area'] = area
    feature['properties']['Population'] = population
    feature['properties']['Dengue_cases'] = dengue_cases
    feature['properties']['Breteau_index'] = BI_entry
    feature['properties']['Breeding_site_counts'] = brd_site_counts
    feature['properties']['Density'] = int(10*brd_site_counts/area)

In [9]:
with open(os.path.join('..', 'data', 'maps', 'Nakhon-Si-Thammarat-choropleth.geojson'), 'w') as FILE:
    json.dump(shapefile, FILE, indent=4, ensure_ascii=False, sort_keys=True)